## Window Theorem (3)

For two separation-identical aircraft $i$ and $j$ where $r_i \le r_j$ and $lt_i \le lt_j$, a *complete-order* can be inferred such that $i$ should precede $j$. Since an aircraft can always be delayed to meet the start of its hard time window, and $t_x \le t'_x$ for all aircraft in corresponding positions of sequences $s$ and $s'$, any time-window violation is no worse in $s$ than in $s'$. Hence, sequencing $i$ before $j$ cannot increase the number or severity of hard time-window violations.


In [4]:

from z3 import *
from itertools import product

s = Solver()


δ  = Function('δ', IntSort(), IntSort(), RealSort())
R  = Function('R', IntSort(), RealSort())
LT = Function('LT', IntSort(), RealSort())

p1, a, p2, b, p3 = Ints('p1 a p2 b p3')
ac = [p1, a, p2, b, p3]

x, y = Ints('x y')

s.add(Distinct(*ac))

s.add(ForAll([x], R(x) >= 0))
s.add(ForAll([x], LT(x) >= 0))
s.add(ForAll([x, y], δ(x, y) >= 0))

# δ-identical a/b
for x_ in ac:
    s.add(δ(a, x_) == δ(b, x_))
    s.add(δ(x_, a) == δ(x_, b))

def zmax(x, y): return If(x >= y, x, y)

def zmax_list(xs):
    m = xs[0]
    for x in xs[1:]:
        m = If(x >= m, x, m)
    return m


def compute_T(seq):
    T = {seq[0]: R(seq[0])}
    for i in range(1, len(seq)):
        preds = seq[:i]
        T[seq[i]] = zmax(R(seq[i]), zmax_list([T[x] + δ(x, seq[i]) for x in preds]))
    return T


S1 = [p1, a, p2, b, p3]
S2 = [p1, b, p2, a, p3]

T1 = compute_T(S1)
T2 = compute_T(S2)

s.add(R(a) <= R(b))
s.add(LT(a) <= LT(b))

def violation(Tx, LTx):  # 1 if Tx > LTx, else 0
    return If(Tx > LTx, 1, 0)

ac_ = [p1, p2, p3]
viol_S1 = Sum([violation(T1[x], LT(x)) for x in ac_])
viol_S2 = Sum([violation(T2[x], LT(x)) for x in ac_])

s.add(viol_S2 < viol_S1)

print("Checking counterexample to Theorem 3 (hard time windows)...")
res = s.check()
print(res)

if res == sat:
    print("\nCounterexample found (would disprove rule):")
    m = s.model()
    for d in m.decls():
        print(f"{d.name()} = {m[d]}".replace("-1", "p1").replace("-2", "a").replace("-3", "p2").replace("-4", "b").replace("-5", "p3"))
else:
    print("\nUNSAT — Theorem 3 holds: a before b cannot worsen hard-window compliance.")

Checking counterexample to Theorem 3 (hard time windows)...
unsat

UNSAT — Theorem 3 holds: a before b cannot worsen hard-window compliance.
